# Getting Started with Apache Iceberg

This notebook provides a comprehensive introduction to Apache Iceberg:

* What is Iceberg?
* Setting up Iceberg
* Creating and managing tables
* Appending data
* Querying tables
* Basic operations

After going through this notebook, you should understand what Iceberg is and how to perform basic table operations.

## What is Apache Iceberg?

Iceberg is a **table format** for huge analytic datasets. Think of it as "Git for data":

* **Parquet** is a file format (like individual source code files)
* **Iceberg** is a table format (like a Git repository that versions those files)

### Key Problems Iceberg Solves

Working with raw Parquet files in a data lake has challenges:

1. **No ACID transactions**: What if your write fails halfway through? You get inconsistent data.
2. **No schema evolution**: Adding a column means rewriting all files.
3. **Slow queries**: Query engines must list and scan all files to find relevant data.
4. **No time travel**: Can't query data as it was yesterday.
5. **Unsafe concurrent writes**: Two writers can corrupt each other's changes.

### How Iceberg Fixes This

Iceberg adds a **metadata layer** on top of Parquet files:

```
Traditional Data Lake          Iceberg Data Lake
┌─────────────────┐           ┌─────────────────┐
│  Parquet Files  │           │ Iceberg Catalog │  ← Points to current metadata
│  (scattered,    │           │                 │
│   untracked)    │           └────────┬────────┘
└─────────────────┘                    │
                               ┌───────▼────────┐
                               │ Metadata JSON  │  ← Schema, snapshots, history
                               └───────┬────────┘
                               ┌───────▼────────┐
                               │ Manifest Files │  ← Lists of data files + stats
                               │   (AVRO)       │
                               └───────┬────────┘
                               ┌───────▼────────┐
                               │ Parquet Files  │  ← Actual data (immutable)
                               │   (tracked,    │
                               │    versioned)  │
                               └────────────────┘
```

This separation of metadata and data provides:

* **ACID transactions**: All-or-nothing commits via atomic metadata updates
* **Schema evolution**: Change schema in metadata without touching data files
* **Fast queries**: Read manifests to find relevant files (no directory listing)
* **Time travel**: Each snapshot points to a specific set of files
* **Safe concurrency**: Optimistic locking detects conflicts

### Iceberg vs. Other Approaches

| Approach | Schema Changes | Time Travel | ACID | Concurrent Writes |
|----------|---------------|-------------|------|------------------|
| **Raw Parquet** | Rewrite all data | Manual snapshots | ❌ | ❌ |
| **Parquet + Hive** | Rewrite all data | Manual snapshots | ❌ | ⚠️ (locking issues) |
| **Iceberg** | Metadata only | Built-in | ✅ | ✅ (optimistic) |
| **Delta Lake** | Metadata only | Built-in | ✅ | ✅ (optimistic) |

*Note: Delta Lake is similar to Iceberg but is more tightly coupled to Spark. Iceberg works with any engine (Daft, Spark, Trino, Flink, etc.)*

### When to Use Iceberg

Use Iceberg when you need:

* **Large-scale analytics**: Billions of rows, millions of files
* **Schema flexibility**: Schema changes without downtime
* **Data governance**: Auditing, versioning, compliance
* **Concurrent access**: Multiple readers and writers
* **Query performance**: Fast queries on partitioned data

Don't use Iceberg for:

* **Small datasets**: < 1 GB - the metadata overhead isn't worth it
* **Single-file scenarios**: Just use Parquet directly
* **Real-time updates**: Iceberg is for batch/micro-batch, not streaming updates

## Setting Up Iceberg

Iceberg requires a **catalog** to manage table metadata. The catalog stores:

* Table locations
* Current metadata file pointers
* Namespace (database) information

We'll use **SqlCatalog** with SQLite for learning purposes. In production, you'd use:

* **AWS Glue Catalog** (for AWS)
* **Nessie** (Git-like catalog with branches)
* **REST Catalog** (generic HTTP-based catalog)
* **Hive Metastore** (for legacy compatibility)

Note: SqlCatalog with SQLite is **not** suitable for production concurrent writes, but it's perfect for learning and development.

In [ ]:
import daft
import pyarrow as pa
import json
from pathlib import Path
from pyiceberg.catalog.sql import SqlCatalog

%reload_ext autoreload
%autoreload 2
from helpers import inspect_iceberg_table

In [ ]:
# Setup paths
warehouse_path = Path('../data/warehouse_getting_started').absolute()
warehouse_path.mkdir(parents=True, exist_ok=True)
catalog_db = warehouse_path / 'catalog.db'

# Clean up previous runs for a fresh start
catalog_db.unlink(missing_ok=True)

# Initialize SQL-based Iceberg catalog
catalog = SqlCatalog(
    'getting_started',
    **{
        'uri': f'sqlite:///{catalog_db}',
        'warehouse': f'file://{warehouse_path}'
    }
)

# Create a namespace (like a database schema)
catalog.create_namespace('iot')

print(f"✅ Catalog initialized at {warehouse_path}")
print(f"   Catalog DB: {catalog_db}")
print(f"   Namespace 'iot' created")

## Creating Tables

There are two main approaches to create Iceberg tables:

1. **Define schema explicitly**: Create table with a PyArrow schema
2. **Infer from data**: Let Iceberg discover the schema from your data

### Approach 1: Explicit Schema

Use this when you know your schema upfront and want strict validation.

In [ ]:
# Define schema explicitly
schema = pa.schema([
    pa.field('device_id', pa.string(), nullable=False),
    pa.field('timestamp', pa.timestamp('ms'), nullable=False),
    pa.field('temperature', pa.float64()),
    pa.field('humidity', pa.float64()),
])

# Create table with explicit schema
sensors_table = catalog.create_table(
    'iot.sensors',
    schema=schema
)

print("✅ Created 'iot.sensors' table with explicit schema")
print(f"   Location: {sensors_table.location()}")

### Approach 2: Infer Schema from Data

Use this when you're working with JSON or other semi-structured data and want quick ingestion.

In [ ]:
# Read sample of events data
jsonl_file = Path('../data/input/events.jsonl')

with jsonl_file.open('r') as f:
    # Read first 50K events
    events_data = [json.loads(line) for i, line in enumerate(f) if i < 50000]

# Convert to Arrow table (schema is auto-discovered)
arrow_table = pa.Table.from_pylist(events_data)

print(f"📊 Loaded {len(events_data):,} events from JSON")
print(f"   Auto-discovered schema: {arrow_table.schema}")

# Create Iceberg table from inferred schema
events_table = catalog.create_table(
    'iot.events',
    schema=pa.schema(arrow_table.schema)
)

print(f"\n✅ Created 'iot.events' table")
print(f"   Location: {events_table.location()}")

### What Got Created?

When you create a table, Iceberg creates this directory structure:

```
warehouse/
└── iot/
    └── events/
        ├── data/         ← Parquet files go here (empty for now)
        └── metadata/     ← Metadata JSON files
```

Let's inspect the warehouse:

In [ ]:
# List the warehouse structure
import os
for root, dirs, files in os.walk(warehouse_path):
    level = root.replace(str(warehouse_path), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files only
        print(f'{subindent}{file}')
    if len(files) > 5:
        print(f'{subindent}... and {len(files) - 5} more files')

## Appending Data

Appending data to an Iceberg table creates a new **snapshot**. Each snapshot is an immutable view of the table at a point in time.

In [ ]:
# Append the events data we loaded earlier
events_table.append(arrow_table)

print(f"✅ Appended {len(events_data):,} records")
print(f"   This created Snapshot 1")

# Check the table history
history = events_table.history()
print(f"\n📜 Table now has {len(history)} snapshot(s)")
for i, snapshot in enumerate(history, 1):
    print(f"   Snapshot {i}: ID {snapshot.snapshot_id}, Timestamp: {snapshot.timestamp_ms}")

### What Happened Behind the Scenes?

When you appended data:

1. **Data file created**: A new Parquet file was written to `data/`
2. **Manifest created**: An AVRO manifest file lists this new data file
3. **Metadata updated**: A new metadata JSON file was created
4. **Catalog updated**: The catalog now points to the new metadata file

This is an **atomic operation** - either all of this happens, or none of it does.

In [ ]:
# Check what files were created
events_dir = Path(events_table.location().replace('file://', ''))
data_files = list(events_dir.glob('data/*.parquet'))
metadata_files = list(events_dir.glob('metadata/*.json'))
manifest_files = list(events_dir.glob('metadata/*.avro'))

print(f"📂 Files created for {len(events_data):,} records:")
print(f"   Parquet data files: {len(data_files)}")
print(f"   Metadata JSON files: {len(metadata_files)}")
print(f"   Manifest AVRO files: {len(manifest_files)}")

if data_files:
    data_size = sum(f.stat().st_size for f in data_files)
    print(f"\n   Total data size: {data_size / 1024 / 1024:.2f} MB")

## Querying Tables

We use **Daft** to read and query Iceberg tables. Daft understands Iceberg's metadata and can perform optimizations like:

* **Column pruning**: Only read the columns you need
* **Predicate pushdown**: Skip files that don't match your filters
* **Partition pruning**: Skip entire partitions (we'll cover this later)

Daft provides two query interfaces:

1. **SQL**: Familiar SQL syntax
2. **DataFrame API**: Similar to Pandas/Spark

In [ ]:
# Read the Iceberg table with Daft
df = daft.read_iceberg(events_table)

# Show schema
print("Schema:")
print(df.schema())

# Show first few rows
print("\nFirst 5 rows:")
df.show(5)

In [ ]:
# SQL query: Count total events
print("Total events:")
daft.sql("""
    SELECT COUNT(*) as total
    FROM df
""").show()

# SQL query: Top event types
print("\nTop 10 event types:")
daft.sql("""
    SELECT type, COUNT(*) as count
    FROM df
    GROUP BY type
    ORDER BY count DESC
    LIMIT 10
""").show()

In [ ]:
# DataFrame API: Same queries as above
print("Using DataFrame API:")

# Top event sources (devices)
(
    df
    .groupby('source')
    .agg(daft.col('source').count().alias('event_count'))
    .sort('event_count', desc=True)
    .limit(10)
    .show()
)

## Basic Operations

Let's explore common table operations:

* **Append**: Add more data
* **Overwrite**: Replace table contents
* **Delete**: Remove records matching a condition

Each operation creates a new snapshot.

### Appending More Data

In [ ]:
# Load next batch of events (next 50K)
with jsonl_file.open('r') as f:
    batch2_data = [json.loads(line) for i, line in enumerate(f) if 50000 <= i < 100000]

batch2_arrow = pa.Table.from_pylist(batch2_data)
events_table.append(batch2_arrow)

print(f"✅ Appended {len(batch2_data):,} more records (Snapshot 2)")

# Verify the count increased
df = daft.read_iceberg(events_table)
print("\nNew total:")
daft.sql("SELECT COUNT(*) as total FROM df").show()

### Deleting Records

Iceberg supports **row-level deletes** using predicates. This creates **delete files** that mark rows as deleted without rewriting the entire data files.

In [ ]:
# Delete all events of a specific type
events_table.delete("type = 'c8y_LocationUpdate'")

print("✅ Deleted 'c8y_LocationUpdate' events (Snapshot 3)")

# Verify they're gone
df = daft.read_iceberg(events_table)
print("\nRemaining events by type:")
daft.sql("""
    SELECT type, COUNT(*) as count
    FROM df
    GROUP BY type
    ORDER BY count DESC
    LIMIT 5
""").show()

### Overwriting Data

Overwrite replaces the entire table contents with new data. This is useful for reprocessing or fixing data issues.

In [ ]:
# Create a small sample table to demonstrate overwrite
sample_table = catalog.create_table(
    'iot.sample',
    schema=pa.schema([pa.field('id', pa.int64()), pa.field('value', pa.string())])
)

# Write initial data
data1 = pa.table({'id': [1, 2, 3], 'value': ['a', 'b', 'c']})
sample_table.append(data1)
print("Initial data:")
daft.read_iceberg(sample_table).show()

# Overwrite with new data
data2 = pa.table({'id': [10, 20], 'value': ['x', 'y']})
sample_table.overwrite(data2)
print("\nAfter overwrite:")
daft.read_iceberg(sample_table).show()

## Inspecting Table Metadata

Let's use our helper function to visualize the complete table structure:

In [ ]:
# Inspect the events table
inspect_iceberg_table(events_table)

## Review Questions

Test your understanding:

1. **What problem does Iceberg solve that Parquet alone doesn't?**
   - Think about transactions, versioning, and metadata management.

2. **Why is metadata separation important for large datasets?**
   - Hint: What happens when you need to list millions of files in S3?

3. **How is Iceberg different from just versioning Parquet files in folders?**
   - Consider: `data/2024-01-01/`, `data/2024-01-02/` vs. Iceberg snapshots

4. **What happens to data files when you delete records?**
   - Are the Parquet files rewritten? What gets created instead?

5. **Why does each operation create a new snapshot?**
   - What would happen without snapshots?

6. **Can you query a table while someone else is writing to it?**
   - Think about snapshot isolation.

## Hands-on Challenge

Now it's your turn! Try these exercises:

### Challenge 1: Create a Table from cmdata.jsonl

1. Load `../data/input/cmdata.jsonl`
2. Create an Iceberg table called `iot.devices`
3. Query the table to find how many devices have alarms

### Challenge 2: Append and Verify

1. Append the first 1000 records from events.jsonl to a new table
2. Append the next 1000 records
3. Verify the total count is 2000
4. Inspect the table history - how many snapshots?

### Challenge 3: Explore Metadata Files

1. Find the metadata JSON file for your events table
2. Open it in a text editor or use `inspect_metadata_json()` (covered in next notebook)
3. Find the `current-snapshot-id` value
4. Can you locate the corresponding snapshot in the `snapshots` array?

Use the cells below for your solution:

In [ ]:
# Challenge 1: Your code here


In [ ]:
# Challenge 2: Your code here


In [ ]:
# Challenge 3: Your code here


## Summary

In this notebook, we covered:

* **What Iceberg is**: A table format that adds ACID transactions, schema evolution, and time travel to data lakes
* **Why it matters**: Solves key problems with raw Parquet files (no transactions, slow queries, no versioning)
* **Setting up**: Using SqlCatalog with SQLite for development
* **Creating tables**: Both explicit schema and inferred from data
* **Appending data**: Each append creates a new immutable snapshot
* **Querying**: Using Daft with SQL and DataFrame API
* **Basic operations**: Append, delete, overwrite - all with ACID guarantees

### Key Takeaways

1. **Metadata is the magic**: Iceberg's metadata layer enables all its features
2. **Snapshots are immutable**: Once created, a snapshot never changes
3. **Operations are atomic**: All-or-nothing commits via catalog updates
4. **Query engines read metadata**: Daft reads manifests to find relevant files
5. **Data files are never edited**: Deletes create delete files, not rewrites

### What's Next?

In the next notebooks, we'll dive deeper into:

* **Metadata structures**: What's inside the JSON and AVRO files?
* **Time travel**: Query historical data, rollback changes
* **Schema evolution**: Add/remove columns without rewrites
* **Concurrency**: How optimistic locking works
* **Partitioning**: Scale to millions of files
* **Object stores**: Iceberg on S3 with predicate pushdown